<a href="https://colab.research.google.com/github/uzer037/graduation_work/blob/dev/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Теория

## Предобработка
(Пока не реализована)
```
HxWx3 - исходное RGB изображение (где min(H,W) >= 512)
| центрирование и обрезка до квадратного разрешения 512x512
v
512x512x3
```
## Описание модели
```
512x512x3
| Conv2d - пытаемся выявить мелкие фичи
v
512x512x3
| avg-pool (kernel 2x2, stride 2) - уменьшаем размер картинки с сохранением деталей
v
256x256x3
| Conv2d - пытаемся выявить средние фичи
v
256x256x3
| max-pool (kernel 4x4, stride 4) - уменьшаем с выделением наиболее сильных фич
v
64x64x3
| Conv2d - пытаемся выявить композиционные фичи
v
64x64x3
| Flatten
v
12288
| Dense | 12288 / 16 = 768 - "сжимаем" результат
v
768
| Dense - Скрытый слой
v
768
| ReLU - Нелинейность
v
768
| Dense - Итоговый классификатор
v
2 - {Выходы: A - реальное фото, B - нейросетевая генерация}
(Возможно потом стоит расширить до n классов, один из которых - реальные фото, а остальные - разные архитектуры генеративных сетей)
```

In [ ]:
# @title Установка
%pip install keras tensorboard
%load_ext tensorboard

In [ ]:
# @title Импорты
import keras
from keras import layers, losses

In [14]:
# @title Объявление модели (draft)
model = keras.Sequential(name="AI_Image_Classifier")
# Layers
model.add(layers.Conv2D(512, (3, 3), input_shape=(512, 512, 3))) # 512x512x3 --> 512x512x3 (fine features)
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2)) # 512x512x3 -(avg)-> 256x256x3
model.add(layers.Conv2D(256, (3, 3))) # 256x256x3 --> 256x256x3 (medium features)
model.add(layers.MaxPooling2D(pool_size=(4, 4), strides=4)) # 256x256x3 -(max)-> 64x64x3
model.add(layers.Conv2D(64, (3, 3))) # 64x64x3 --> 64x64x3 (global features)
model.add(layers.Flatten()) # 64x64x3 --> 12288
model.add(layers.Dense(768)) # 12288 --> 768 (= 12288 / 16)
model.add(layers.Activation('relu'))
model.add(layers.Dense(2)) # 768 --> 2 classes
model.add(layers.Activation('softmax'))
# Loss
model.add_loss(losses.CategoricalCrossentropy())


model.summary()

Model: "AI_Image_Classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 510, 510, 512)     14336     
                                                                 
 average_pooling2d_3 (Avera  (None, 255, 255, 512)     0         
 gePooling2D)                                                    
                                                                 
 conv2d_10 (Conv2D)          (None, 253, 253, 256)     1179904   
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 63, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 61, 61, 64)        147520    
                                                                 
 flatten_3 (Flatten)         (None, 238144)    

In [ ]:
# @title Tensorboard
%tensorboard --logdir runs